In [ ]:
# Josh Gilkinson

In [ ]:
# DSC 630 Week 3 Assignment 

In [1]:
import pandas as pd

Dodgers_path = r'C:\Users\jgilk\OneDrive\Desktop\Data Science\DSC630\dodgers-2022.csv'
df = pd.read_csv(Dodgers_path)


In [26]:
# Sort by attendance 
top_10_games = df.sort_values(by='attend', ascending=False).head(10)

print("Top 10 Games Based on Attendance:")
print(top_10_games)


Top 10 Games Based on Attendance:
   month  day  attend day_of_week   opponent  temp   skies day_night cap  \
0    APR   10   56000     Tuesday    Pirates    67  Clear        Day  NO   
59   AUG   21   56000     Tuesday     Giants    75  Clear      Night  NO   
39   JUL    1   55359      Sunday       Mets    75  Clear      Night  NO   
31   JUN   12   55279     Tuesday     Angels    66  Cloudy     Night  NO   
56   AUG    7   55024     Tuesday    Rockies    80  Clear      Night  NO   
64   AUG   30   54621    Thursday     Snakes    80  Clear      Night  NO   
10   APR   28   54242    Saturday  Nationals    71  Clear      Night  NO   
44   JUL   14   54014    Saturday     Padres    75  Clear      Night  NO   
42   JUL    4   53570   Wednesday       Reds    70  Clear      Night  NO   
35   JUN   17   53504      Sunday  White Sox    74  Clear        Day  NO   

   shirt fireworks bobblehead  
0     NO        NO         NO  
59    NO        NO        YES  
39    NO        NO        YES  
3

Is there a correlation between attendance and promotional gifts (Hats, Bobble Heads, Shirts, Fireworks Displays)

In [19]:
# Creating binary columns for promo items
promotion_columns = ['cap', 'shirt', 'fireworks', 'bobblehead']
for col in promotion_columns:
    df[col] = df[col].apply(lambda x: 1 if x == 'YES' else 0)

promotion_correlation = df[['attend'] + promotion_columns].corr()['attend']

# Drop the Attend Column
promotion_correlation_without_attend = promotion_correlation.drop('attend')
promotion_correlation_sorted = promotion_correlation_without_attend.sort_values(ascending=False)

print("Correlation Between Attendance and Promotional Items:")
for promo, correlation in promotion_correlation_sorted.items():
    print(f"{promo:<15} {correlation:.6f}")

Correlation Between Attendance and Promotional Items:
bobblehead      0.581895
shirt           0.133269
fireworks       0.002094
cap             -0.055002


These numbers confirm what the initial review hinted at, the bobble head is far and away the best promotional item. The 0.58 correlation is considered the upper end of the Moderate Correlation chart. T-shirts a distant second place with only a very weak correlation, firework displays are another very weak correlation and also they cost the same regardless of attendees. If there is 100 or 10,000 the show costs the same. Finally, baseball caps actually appear to have a negative correlation however this could be due to a limited sample size, but it is something to consider. 

Is there a correlation between attendance and opponent

In [15]:
# Convert opponent column into dummy data
opponent_dummies = pd.get_dummies(df['opponent'], prefix='opponent')

# Add opponent dummy data
df_with_dummies = pd.concat([df, opponent_dummies], axis=1)

opponent_correlation = df_with_dummies[['attend'] + opponent_dummies.columns.tolist()].corr()['attend']

# Drop the Attend column
opponent_correlation_without_attend = opponent_correlation.drop('attend')
opponent_correlation_without_attend.index = opponent_correlation_without_attend.index.str.replace('opponent_', '')

# Sorting the data
opponent_correlation_sorted = opponent_correlation_without_attend.sort_values(ascending=False)

print("Correlation Between Attendance and Opponents:")
for opponent, correlation in opponent_correlation_sorted.items():
    print(f"{opponent:<15} {correlation:.6f}")

Correlation Between Attendance and Opponents:
Mets            0.236213
Angels          0.207796
Nationals       0.195667
White Sox       0.127046
Cubs            0.075310
Padres          0.045111
Phillies        0.020380
Cardinals       -0.006967
Marlins         -0.008912
Reds            -0.009301
Rockies         -0.060404
Pirates         -0.071849
Snakes          -0.073943
Giants          -0.074763
Astros          -0.134533
Brewers         -0.157030
Braves          -0.209171


Honestly, this was the most suprising to me. I had expected some stronger correlations between opponent and attendance but that really wasn't there. I had thought location might matter, other California teams, but the Padres and Giants have very weak correlations. Only the Angels have a slightly weak correlation and are based near the Dodgers. Alternatively, I went online to discover records from 2022 to see if ranking could have a factor. Both the Mets and the Braves had a win pct of .623 and the Mets had the best positive correlation and the Braves had the best Negative correlation. It should be noted the Dodgers had a win pct of .685, better than both teams, so it is unclear based on this data why people actively avoided the Braves and went to see the Mets. 

Season Order

In [21]:
df['game_number'] = range(1, len(df) + 1)

correlation_game_number_attendance = df[['attend', 'game_number']].corr().loc['attend', 'game_number']

# Print the result
print("Correlation Between Attendance and Season Order:")
print(f"Game Number and Attendance: {correlation_game_number_attendance:.6f}")

Correlation Between Attendance and Season Order:
Game Number and Attendance: 0.057033


This shows there is a very week correlation between season order and attendance. There are multiple possible reasons for this and it should not be considered unless in a playoff situation. 

Day vs Night

In [8]:
df = df.iloc[1:]  # Drop the first game

df['day_night_binary'] = df['day_night'].apply(lambda x: 1 if x == 'Day' else 0)

correlation_day_night = df[['attend', 'day_night_binary']].corr().loc['attend', 'day_night_binary']

avg_attendance_day = df[df['day_night'] == 'Day']['attend'].mean()
avg_attendance_night = df[df['day_night'] == 'Night']['attend'].mean()

print("Correlation Between Attendance and Time of Day (Excluding First Game):")
print(f"Correlation with Day Games: {correlation_day_night:.6f}")

print("\nAverage Attendance (Excluding First Game):")
print(f"Day Games       {avg_attendance_day:.2f}")
print(f"Night Games     {avg_attendance_night:.2f}")

Correlation Between Attendance and Time of Day (Excluding First Game):
Correlation with Day Games: -0.012373

Average Attendance (Excluding First Game):
Day Games       40778.50
Night Games     41040.28


The average night game attendance is only slightly higher than the average day attendance. I omitted the first game of the season as it was an outlier that would skew the data. The opening game was omitted from all other "Day" analysis. 

Day Games: Weekend vs Weekday

In [9]:
df = df.iloc[1:]  # Drop the first game 

df['weekday_weekend'] = df['day_of_week'].apply(
    lambda x: 'Weekend' if x in ['Friday', 'Saturday', 'Sunday'] else 'Weekday'
)

day_games = df[df['day_night'] == 'Day']

avg_attendance_day_weekday = day_games[day_games['weekday_weekend'] == 'Weekday']['attend'].mean()
avg_attendance_day_weekend = day_games[day_games['weekday_weekend'] == 'Weekend']['attend'].mean()

print("Average Attendance for Day Games by Weekday/Weekend (Excluding First Game):")
print(f"Weekday Day Games: {avg_attendance_day_weekday:.2f}")
print(f"Weekend Day Games: {avg_attendance_day_weekend:.2f}")

Average Attendance for Day Games by Weekday/Weekend (Excluding First Game):
Weekday Day Games: 38275.50
Weekend Day Games: 41195.67


By breaking down the Day game average even further, we can see weekday games fall behind weekend games 

In [ ]:
Night Games: Weekend vs Weekday

In [10]:
df['weekday_weekend'] = df['day_of_week'].apply(
    lambda x: 'Weekend' if x in ['Friday', 'Saturday', 'Sunday'] else 'Weekday'
)

night_games = df[df['day_night'] == 'Night']

avg_attendance_night_weekday = night_games[night_games['weekday_weekend'] == 'Weekday']['attend'].mean()
avg_attendance_night_weekend = night_games[night_games['weekday_weekend'] == 'Weekend']['attend'].mean()

print("Average Attendance for Night Games by Weekday/Weekend:")
print(f"Weekday Night Games: {avg_attendance_night_weekday:.2f}")
print(f"Weekend Night Games: {avg_attendance_night_weekend:.2f}")

Average Attendance for Night Games by Weekday/Weekend:
Weekday Night Games: 40612.84
Weekend Night Games: 42096.85


A further breakdown of Nightgames shows a reasonable increase in attendence for weekday day games but only a minor increase over weekend day games 

Since night games and bobbleheads have been shown to contribute heavely to the attendance, how do they interact with each other. 

In [5]:
night_games = df[df['day_night'] == 'Night']

# Break out the bobblehead games
night_games_with_bobblehead = night_games[night_games['bobblehead'] == 'YES']
night_games_without_bobblehead = night_games[night_games['bobblehead'] == 'NO']

avg_attendance_with_bobblehead = night_games_with_bobblehead['attend'].mean()
avg_attendance_without_bobblehead = night_games_without_bobblehead['attend'].mean()

print(f"Average Attendance for Night Games with Bobblehead: {avg_attendance_with_bobblehead:.2f}")
print(f"Average Attendance for Night Games without Bobblehead: {avg_attendance_without_bobblehead:.2f}")


Average Attendance for Night Games with Bobblehead: 53144.64
Average Attendance for Night Games without Bobblehead: 38413.75


Based on this, night games alone are not enough to increase ticket sales. Bobbleheads will add a significant amount to the ticket sales. 

Final Suggestions:

Based on all the reviews performed the best way to increase ticket sales would be to hold a night game on the weekend and give out bobblehead figures. However, since the ticket sales are capped at 56,000, it would be a better idea to save the bobbleheads for lower performing situations like day games during the work week to offset the expected drop in sales. 

Refrences for baseball standings 
https://www.mlb.com/standings/2022